In [ ]:
import numpy as np
from transformers import pipeline

In [ ]:
p = pipeline('feature-extraction')

In [ ]:
np.array(p(["Hellow word!", "Hellow word!"])).shape

In [1]:
import os
from pathlib import Path

In [2]:
IMPRESSIONV2_DIR = Path('/impressionv2')

In [3]:
dirs = os.listdir(IMPRESSIONV2_DIR/'train')[:10]

In [4]:
os.listdir(IMPRESSIONV2_DIR/'train'/dirs[0])

['spectrograms',
 'audio_segments',
 'fi_face_resnet18',
 'openface',
 'pyaa',
 'egemaps',
 'YbxdeEd7wq4.001.wav',
 'frames',
 'dynimgs']

In [5]:
IMPRESSIONV2_DIR/'train'/dirs[0]/'YbxdeEd7wq4.001.wav'

PosixPath('/impressionv2/train/YbxdeEd7wq4.001/YbxdeEd7wq4.001.wav')

In [14]:
import soundfile as sf
import torch
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC, Wav2Vec2Model

# load pretrained model
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
raw_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [15]:
audio_input, sampling_rate = sf.read(IMPRESSIONV2_DIR/'train'/dirs[0]/'YbxdeEd7wq4.001.wav')

In [16]:
from torchaudio.transforms import Resample
r = Resample(sampling_rate)
audio_input = r.forward(torch.tensor(audio_input))

In [17]:
input_values = tokenizer(audio_input, return_tensors="pt", padding="longest").input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.decode(predicted_ids[0])

In [18]:
states = raw_model(input_values)

In [33]:
audio_input.shape, input_values.shape, logits.shape, predicted_ids.shape

(torch.Size([244832]),
 torch.Size([1, 244832]),
 torch.Size([1, 764, 32]),
 torch.Size([1, 764]),
 268)

In [35]:
model.linear

ModuleAttributeError: 'Wav2Vec2ForCTC' object has no attribute 'linear'

In [22]:
states.last_hidden_state.shape

torch.Size([1, 764, 768])

In [26]:
from torchsummary import summary

In [30]:
summary(model)

Layer (type:depth-idx)                                  Param #
├─Wav2Vec2Model: 1-1                                    --
|    └─Wav2Vec2FeatureExtractor: 2-1                    --
|    |    └─ModuleList: 3-1                             4,200,448
|    └─Wav2Vec2FeatureProjection: 2-2                   --
|    |    └─LayerNorm: 3-2                              1,024
|    |    └─Linear: 3-3                                 393,984
|    |    └─Dropout: 3-4                                --
|    └─Wav2Vec2Encoder: 2-3                             --
|    |    └─Wav2Vec2PositionalConvEmbedding: 3-5        4,719,488
|    |    └─LayerNorm: 3-6                              1,536
|    |    └─Dropout: 3-7                                --
|    |    └─ModuleList: 3-8                             85,054,464
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           24,608
Total params: 94,395,552
Trainable params: 94,395,552
Non-trainable param

Layer (type:depth-idx)                                  Param #
├─Wav2Vec2Model: 1-1                                    --
|    └─Wav2Vec2FeatureExtractor: 2-1                    --
|    |    └─ModuleList: 3-1                             4,200,448
|    └─Wav2Vec2FeatureProjection: 2-2                   --
|    |    └─LayerNorm: 3-2                              1,024
|    |    └─Linear: 3-3                                 393,984
|    |    └─Dropout: 3-4                                --
|    └─Wav2Vec2Encoder: 2-3                             --
|    |    └─Wav2Vec2PositionalConvEmbedding: 3-5        4,719,488
|    |    └─LayerNorm: 3-6                              1,536
|    |    └─Dropout: 3-7                                --
|    |    └─ModuleList: 3-8                             85,054,464
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           24,608
Total params: 94,395,552
Trainable params: 94,395,552
Non-trainable param

In [28]:
summary(raw_model)

Layer (type:depth-idx)                             Param #
├─Wav2Vec2FeatureExtractor: 1-1                    --
|    └─ModuleList: 2-1                             --
|    |    └─Wav2Vec2GroupNormConvLayer: 3-1        6,144
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-2      786,432
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-3      786,432
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-4      786,432
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-5      786,432
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-6      524,288
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-7      524,288
├─Wav2Vec2FeatureProjection: 1-2                   --
|    └─LayerNorm: 2-2                              1,024
|    └─Linear: 2-3                                 393,984
|    └─Dropout: 2-4                                --
├─Wav2Vec2Encoder: 1-3                             --
|    └─Wav2Vec2PositionalConvEmbedding: 2-5        --
|    |    └─Conv1d: 3-8                            4,719,488
|    |    └─Wav2Vec2SamePadLa

Layer (type:depth-idx)                             Param #
├─Wav2Vec2FeatureExtractor: 1-1                    --
|    └─ModuleList: 2-1                             --
|    |    └─Wav2Vec2GroupNormConvLayer: 3-1        6,144
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-2      786,432
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-3      786,432
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-4      786,432
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-5      786,432
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-6      524,288
|    |    └─Wav2Vec2NoLayerNormConvLayer: 3-7      524,288
├─Wav2Vec2FeatureProjection: 1-2                   --
|    └─LayerNorm: 2-2                              1,024
|    └─Linear: 2-3                                 393,984
|    └─Dropout: 2-4                                --
├─Wav2Vec2Encoder: 1-3                             --
|    └─Wav2Vec2PositionalConvEmbedding: 2-5        --
|    |    └─Conv1d: 3-8                            4,719,488
|    |    └─Wav2Vec2SamePadLa

In [10]:
transcription

'BROTHERS MARIETS WEK GOT SPENDTIME WITH MY SISTER IN LAW IS WELL AND I WAS ACTUALLY SUPPOSED TO SPEND ON SOME ADDITIONAL TIME AN ELE AH THAT WAS WHERE MY EUTO NETWORK IS BASED OUT OF AND SO I PLANNED AN TO GO VISIT TH OFFICES AND HAVE LUNCH WITH SOME PEOPLE INSTAFF ON'

In [ ]:
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import soundfile as sf
import torch
from jiwer import wer


librispeech_eval = load_dataset("librispeech_asr", "clean", split="test")

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to("cuda")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

librispeech_eval = librispeech_eval.map(map_to_array)

def map_to_pred(batch):
    input_values = tokenizer(batch["speech"], return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        logits = model(input_values.to("cuda")).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    batch["transcription"] = transcription
    return batch

result = librispeech_eval.map(map_to_pred, batched=True, batch_size=1, remove_columns=["speech"])

print("WER:", wer(result["text"], result["transcription"]))